 # 1. Setup Watson Machine Learning Connection

In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
PROJECT_ID = 'xxx4-2daee35ddda6'
API_KEY = 'Cpk3k0Sz8T96MesIzlFTtq-O8xKuQPiVs4gPqaN_ZusH'
LOCATION = 'https://us-south.ml.cloud.ibm.com'

In [8]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "apikey": API_KEY,
    "url": LOCATION
}

wml_client = APIClient(wml_credentials)

wml_client.set.default_space('d15f4a9e-e1f5-4bc2-ae89-6606bd2b62c8')


In [9]:
wml_client.set.default_project(PROJECT_ID)

'SUCCESS'

# 2. Setting up DB2 Connection

In [10]:
import pandas as pd
import tensorflow as tf
import keras


In [11]:
# This connection object is used to access your data and contains your credentials or project token.
# You might want to remove those credentials before you share your notebook.


import os, ibm_db, ibm_db_dbi as dbi, pandas as pd

MLops_db2_connection_dsn = 'DATABASE={};HOSTNAME={};PORT={};PROTOCOL=TCPIP;UID={uid};PWD={pwd};SECURITY=SSL'.format(
    'bludb',
    'b70af05b-76e4-4bca-a1f5-23dbb4c6a74e.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud',
    32716,
    uid='vbz23877',
    pwd="""N2If7ScvYPWfoA7h"""
)

MLops_db2_connection_connection = dbi.connect(MLops_db2_connection_dsn)
   
query = 'SELECT * FROM "VBZ23877"."BANKCHURNERS_TABLE"'

df = pd.read_sql_query(query, con=MLops_db2_connection_connection)
df.head()

# After use, close the database connection with the following code:
# MLops_db2_connection_connection.close()


,CLIENTNUM,ATTRITION_FLAG,CUSTOMER_AGE,GENDER,DEPENDENT_COUNT,EDUCATION_LEVEL,MARITAL_STATUS,INCOME_CATEGORY,CARD_CATEGORY,MONTHS_ON_BOOK,...,MONTHS_INACTIVE_12_MON,CONTACTS_COUNT_12_MON,CREDIT_LIMIT,TOTAL_REVOLVING_BAL,AVG_OPEN_TO_BUY,TOTAL_AMT_CHNG_Q4_Q1,TOTAL_TRANS_AMT,TOTAL_TRANS_CT,TOTAL_CT_CHNG_Q4_Q1,AVG_UTILIZATION_RATIO
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


# 3. Prepare the data

In [12]:
# convert all float columns to int
def convert_float_to_int(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('int64')
    return df

df=convert_float_to_int(df) 

In [14]:
# ignoring all non numeric features for now (just to keep it simple)
numeric_feature_names = ['CUSTOMER_AGE', 'DEPENDENT_COUNT', 'MONTHS_ON_BOOK',  'TOTAL_RELATIONSHIP_COUNT', 'MONTHS_INACTIVE_12_MON','CONTACTS_COUNT_12_MON','CREDIT_LIMIT','TOTAL_REVOLVING_BAL','AVG_OPEN_TO_BUY','TOTAL_AMT_CHNG_Q4_Q1','TOTAL_TRANS_CT','TOTAL_CT_CHNG_Q4_Q1','AVG_UTILIZATION_RATIO']
numeric_features = df[numeric_feature_names]
numeric_features.head()

,CUSTOMER_AGE,DEPENDENT_COUNT,MONTHS_ON_BOOK,TOTAL_RELATIONSHIP_COUNT,MONTHS_INACTIVE_12_MON,CONTACTS_COUNT_12_MON,CREDIT_LIMIT,TOTAL_REVOLVING_BAL,AVG_OPEN_TO_BUY,TOTAL_AMT_CHNG_Q4_Q1,TOTAL_TRANS_CT,TOTAL_CT_CHNG_Q4_Q1,AVG_UTILIZATION_RATIO
0,45,3,39,5,1,3,12691,777,11914,1,42,1,0
1,49,5,44,6,1,2,8256,864,7392,1,33,3,0
2,51,3,36,4,1,0,3418,0,3418,2,20,2,0
3,40,4,34,3,4,1,3313,2517,796,1,20,2,0
4,40,3,21,5,1,0,4716,0,4716,2,28,2,0


In [17]:
train_dataset = numeric_features.sample(frac=0.8, random_state=0)
test_dataset = numeric_features.drop(train_dataset.index)

train_target = train_dataset.pop('CREDIT_LIMIT')
test_target = test_dataset.pop('CREDIT_LIMIT')

In [63]:
# This generally is a good idea, but not using a normalizer for now since it causes issues when saving the model https://github.com/keras-team/keras/issues/15348

# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(train_dataset)

# 4. Setting Up Model Monitoring

In [22]:
training_data_references = [
                {
                    "id": "credit_limit_prediction",
                    "type": "container",
                    "connection": {},
                    "location": {
                        "path": "customer_churn_db2"
                    },
                    "schema": {
                        "id": "training_schema",
                        "fields": [
                            {"name": "I_SHOULD_WRITE_A_SCRIPT_TO_AUTOMATE_THIS", "nullable": True, "metadata": {}, "type": "string"}
                        ]
                    }
                }
            ]

In [23]:
try:
    from ibm_aigov_facts_client import AIGovFactsClient
except:
    !pip install -U ibm-aigov-facts-client
    from ibm_aigov_facts_client import AIGovFactsClient
    
PROJECT_UID= os.environ['PROJECT_ID']
CPD_URL=os.environ['RUNTIME_ENV_APSX_URL'][len('https://api.'):]
CONTAINER_ID=PROJECT_ID
CONTAINER_TYPE='project'
EXPERIMENT_NAME='credit_limit_prediction'

PROJECT_ACCESS_TOKEN=project.project_context.accessToken.replace('Bearer ','')

facts_client = AIGovFactsClient(api_key=API_KEY,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

     |████████████████████████████████| 106 kB 29.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 547 kB 96.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 182 kB 43.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 2.9 MB/s  eta 0:00:01
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139103 sha256=3b7a89792908f4c0390dc9afac841e7f6d8523f0944efc856db84becd030a78c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/7b/ef/c5/85718fa9e66dec117e42d8b4d7b8a2e40ebdec17232935615f
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5028 sha256=2d8bb6a86e93867965380df4dc1e2594d70996b0d7a3347337b7d66eaca8ff16
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/5d/45/1d/a7d2bf

# 5. Do the actual Machine Learning

 https://www.tensorflow.org/tutorials/keras/regression

In [61]:
# not using a normalizer since it causes issues when saving the model https://github.com/keras-team/keras/issues/15348
def get_basic_model():
  model = tf.keras.Sequential([
    #normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['accuracy'])
  return model

In [62]:
model = get_basic_model()
model.fit(train_dataset, train_target, epochs=100, batch_size=50, validation_split = 0.2)

Epoch 1/300
130/130 [==============================] - 1s 5ms/step - loss: 191990464.0000 - accuracy: 0.0000e+00 - val_loss: 170929808.0000 - val_accuracy: 0.0000e+00
Epoch 2/300
130/130 [==============================] - 0s 3ms/step - loss: 116546376.0000 - accuracy: 0.0000e+00 - val_loss: 43249500.0000 - val_accuracy: 0.0000e+00
Epoch 3/300
130/130 [==============================] - 0s 3ms/step - loss: 7366735.5000 - accuracy: 0.0000e+00 - val_loss: 613370.6875 - val_accuracy: 0.0000e+00
Epoch 4/300
130/130 [==============================] - 0s 3ms/step - loss: 184550.5469 - accuracy: 0.0000e+00 - val_loss: 4817.1958 - val_accuracy: 0.0000e+00
Epoch 5/300
130/130 [==============================] - 0s 3ms/step - loss: 2090.3406 - accuracy: 0.0000e+00 - val_loss: 1359.3800 - val_accuracy: 0.0000e+00
Epoch 6/300
130/130 [==============================] - 0s 3ms/step - loss: 1160.5482 - accuracy: 0.0000e+00 - val_loss: 1114.4781 - val_accuracy: 0.0000e+00
Epoch 7/300
130/130 [===========

In [64]:
test_dataset.iloc[0]

CUSTOMER_AGE                   45
DEPENDENT_COUNT                 3
MONTHS_ON_BOOK                 39
TOTAL_RELATIONSHIP_COUNT        5
MONTHS_INACTIVE_12_MON          1
CONTACTS_COUNT_12_MON           3
TOTAL_REVOLVING_BAL           777
AVG_OPEN_TO_BUY             11914
TOTAL_AMT_CHNG_Q4_Q1            1
TOTAL_TRANS_CT                 42
TOTAL_CT_CHNG_Q4_Q1             1
AVG_UTILIZATION_RATIO           0
Name: 0, dtype: int64

In [70]:
index=10
print(model.predict(test_dataset.iloc[[1]]))
print(test_target.iloc[[1]])

[[8546.831]]
13    8547
Name: CREDIT_LIMIT, dtype: int64


# 6. Storing the model

WTF? https://stackoverflow.com/questions/53258236/how-to-save-keras-model-in-wml-repository-in-watson-studio

Do this: 
https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd3.5/notebooks/python_sdk/deployments/keras/Use%20Keras%20to%20recognize%20hand-written%20digits.ipynb

In [71]:
model_result_path = "keras_model.h5"
model.save(model_result_path)

In [72]:
!tar -zcvf keras_model.tgz keras_model.h5

keras_model.h5


In [73]:
! ls -la

total 72
drwxr-x--- 6 wsuser wscommon  4096 Dec  5 12:43 .
drwxrwx--- 1 wsuser wscommon  4096 Dec  5 12:32 ..
drwxr-x--- 4 wsuser wscommon  4096 Dec  5 12:43 keras_model
-rw-rw---- 1 wsuser wscommon 35432 Dec  5 12:52 keras_model.h5
-rw-rw---- 1 wsuser wscommon  5508 Dec  5 12:52 keras_model.tgz
drwxrwx--- 2 wsuser wscommon  4096 Dec  5 12:32 logs
drwxrwx--- 4 wsuser wscommon  4096 Dec  5 12:32 mlruns
drwxr-x--- 2 wsuser wscommon  4096 Dec  5 12:25 .virtual_documents


## 6.1. upload the model

In [74]:
! pip list | grep tensorflow # get the tensorflow version for the model metadata 

tensorflow                    2.7.2
tensorflow-addons             0.15.0
tensorflow-estimator          2.7.0
tensorflow-hub                0.12.0
tensorflow-io-gcs-filesystem  0.23.1
tensorflow-metadata           1.5.0
tensorflow-probability        0.15.0
tensorflow-text               2.7.3


In [75]:
sofware_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")

metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'credit limit model',
            wml_client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = wml_client.repository.store_model(
    model="keras_model.tgz",
    meta_props=metadata)

## 6.2. get the model information programatically

In [76]:
import json

published_model_uid = wml_client.repository.get_model_id(published_model)
model_details = wml_client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "12b83a17-24d8-5082-900f-0ab31fbfd3cb",
      "name": "runtime-22.1-py3.9"
    },
    "type": "tensorflow_2.7"
  },
  "metadata": {
    "created_at": "2022-12-05T12:53:02.805Z",
    "id": "ad59ee55-7103-47cd-99a1-ff5a3a6cf1c2",
    "modified_at": "2022-12-05T12:53:05.270Z",
    "name": "credit limit model",
    "owner": "IBMid-6650024HXN",
    "resource_key": "d916159f-af8d-41c5-8fd9-7dada48aa1c0",
    "space_id": "d15f4a9e-e1f5-4bc2-ae89-6606bd2b62c8"
  },
  "system": {
    "warnings": []
  }
}


In [77]:
models_details = wml_client.repository.list_models()


------------------------------------  ------------------  ------------------------  --------------
ID                                    NAME                CREATED                   TYPE
ad59ee55-7103-47cd-99a1-ff5a3a6cf1c2  credit limit model  2022-12-05T12:53:02.002Z  tensorflow_2.7
885da41b-b808-421e-9f0e-49cc0d5f6da6  credit limit model  2022-12-05T12:44:10.002Z  tensorflow_2.7
d2569dd9-1299-4846-af65-602d78432cca  credit limit model  2022-12-05T12:37:08.002Z  tensorflow_2.7
951eb7fb-8880-4a67-a066-63e2e26be830  credit limit model  2022-12-05T09:15:29.002Z  tensorflow_2.7
712dca3b-ff08-4e76-b7be-5bbedcd5851b  credit limit model  2022-12-05T07:56:16.002Z  tensorflow_2.7
ea57b539-8d0c-4aed-b1e7-5402813dde05  credit limit model  2022-12-05T06:13:18.002Z  tensorflow_2.7
------------------------------------  ------------------  ------------------------  --------------


In [78]:

metadata = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "Deployment of external Keras model",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = wml_client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'ad59ee55-7103-47cd-99a1-ff5a3a6cf1c2' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='081870c8-c1df-479f-8d5f-4634c3defafe'
------------------------------------------------------------------------------------------------




In [80]:
deployment_uid = wml_client.deployments.get_id(created_deployment)

In [82]:
scoring_endpoint = wml_client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/081870c8-c1df-479f-8d5f-4634c3defafe/predictions


In [83]:
test_dataset.iloc[[1]]

,CUSTOMER_AGE,DEPENDENT_COUNT,MONTHS_ON_BOOK,TOTAL_RELATIONSHIP_COUNT,MONTHS_INACTIVE_12_MON,CONTACTS_COUNT_12_MON,TOTAL_REVOLVING_BAL,AVG_OPEN_TO_BUY,TOTAL_AMT_CHNG_Q4_Q1,TOTAL_TRANS_CT,TOTAL_CT_CHNG_Q4_Q1,AVG_UTILIZATION_RATIO
13,35,3,30,5,1,3,1666,6881,1,33,2,0


In [92]:
scoring_payload = {"input_data": [{"values": test_dataset.iloc[[2]]}]}
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
predictions['predictions'][0]['values'][0][0][0] #pretty weird data structure :-P 

1438.128662109375